##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFLite 创作工具

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/guide/authoring"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>
</td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/guide/authoring.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/guide/authoring.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/guide/authoring.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>

TensorFlow Lite Authoring API 提供了一种方式来维护与 TensorFlow Lite 兼容的 `tf.function` 模型。


## 安装

In [ ]:
import tensorflow as tf

## TensorFlow 到 TensorFlow Lite 的兼容性问题

如果您想在设备端使用您的 TF 模型，您需要将其转换为 TFLite 模型，以便从 TFLite 解释器使用它。在转换过程中，您可能会遇到兼容性错误，因为 TFLite 内置运算集不支持 TensorFlow 运算。

这是一个令人讨厌的问题。如何能够更早地检测到该问题呢，比如在模型创作时间？

请注意，以下代码将在调用 `converter.convert()` 时失败。


In [ ]:
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
result = f(tf.constant([0.0]))
print (f"result = {result}")

In [ ]:
# Convert the tf.function
converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [f.get_concrete_function()], f)
try:
  fb_model = converter.convert()
except Exception as e:
  print(f"Got an exception: {e}")

## 简单的目标感知创作用法

我们引入了 Authoring API 来检测模型创作期间的 TensorFlow Lite 兼容性问题。

您只需添加 `@tf.lite.experimental.authoring.compatible` 修饰器来封装您的 `tf.function` 模型，以检查 TFLite 兼容性。

之后，当您评估模型时，将自动检查兼容性。

In [ ]:
@tf.lite.experimental.authoring.compatible
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
result = f(tf.constant([0.0]))
print (f"result = {result}")


如果发现任何 TensorFlow Lite 兼容性问题，它将显示 `COMPATIBILITY WARNING` 或 `COMPATIBILITY ERROR` 以及有问题的运算的确切位置。在本例中，它显示了 tf.unction 模型中 `tf.Cosh` 运算的位置。

您还可以使用 `<function_name>.get_compatibility_log()` 方法检查兼容性日志。

In [ ]:
compatibility_log = '\n'.join(f.get_compatibility_log())
print (f"compatibility_log = {compatibility_log}")

## 引发不兼容性异常

您可以为 `@tf.lite.experimental.authoring.compatible` 修饰器提供一个选项。当您尝试评估经过修饰的模型时，`raise_exception` 选项会引发异常。

In [ ]:
@tf.lite.experimental.authoring.compatible(raise_exception=True)
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
try:
  result = f(tf.constant([0.0]))
  print (f"result = {result}")
except Exception as e:
  print(f"Got an exception: {e}")

## 指定 "Select TF ops" 用法

如果您已了解 [Select Tf op](https://www.tensorflow.org/lite/guide/ops_select) 用法，可以通过设置 `converter_target_spec` 将其用于 Authoring API。它与您将在 [tf.lite.TFLiteConverter](https://www.tensorflow.org/api_docs/python/tf/lite/TFLiteConverter) API 中使用的对象相同。


In [ ]:
target_spec = tf.lite.TargetSpec()
target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
@tf.lite.experimental.authoring.compatible(converter_target_spec=target_spec, raise_exception=True)
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
result = f(tf.constant([0.0]))
print (f"result = {result}")


## 检查 GPU 兼容性

如果您希望确保您的模型与 TensorFlow Lite 的 [GPU 委托](https://www.tensorflow.org/lite/performance/gpu)兼容，您可以设置 [tf.lite.TargetSpec](https://www.tensorflow.org/api_docs/python/tf/lite/TargetSpec) 的 `experimental_supported_backends`。

以下示例展示如何确保您的模型的 GPU 委托兼容性。请注意，此模型存在兼容性问题，因为它使用带有 tf.slice 算子和不受支持的 tf.cosh 算子的二维张量。您将看到两个带有位置信息的 `COMPATIBILITY WARNING`。

In [ ]:
target_spec = tf.lite.TargetSpec()
target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
target_spec.experimental_supported_backends = ["GPU"]
@tf.lite.experimental.authoring.compatible(converter_target_spec=target_spec)
@tf.function(input_signature=[
    tf.TensorSpec(shape=[4, 4], dtype=tf.float32)
])
def func(x):
  y = tf.cosh(x)
  return y + tf.slice(x, [1, 1], [1, 1])

result = func(tf.ones(shape=(4,4), dtype=tf.float32))

## 阅读更多

有关更多信息，请参阅：

- [tf.function](https://www.tensorflow.org/api_docs/python/tf/function) API 文档
- [使用 tf.function 提升性能](https://www.tensorflow.org/guide/function)
- [TensorFlow Lite Converter](https://www.tensorflow.org/lite/models/convert)
- [TensorFlow Lite Model Analyzer](https://www.tensorflow.org/lite/guide/model_analyzer)
- [TensorFlow Lite GPU 委托](https://www.tensorflow.org/lite/performance/gpu)